<h1>Interacting with the VAMPS-API</h1>
<h2>This notebook:</h2>
<ul>
<li>Logs into VAMPS account, if you don't have an account, use guest account (Username: 'guest', Password: 'guest')</li>
<li>Produces a config file for MVCO eukaryote data, selecting the Alpha Diversity visualization</li>
<li>Displays visualization using configuration file</li>
<li>Takes the same configuration file, modifies the project to be for MVCO bacteria data</li>
<li>Displays same vizualization using same constraints, with the different project</li>
</ul>
<h2>To use this notebook:</h2>
<ul><li>Press run cell button</li>
<li>Enter information if promtped</li>
<li>Do not run the next cell if the previous cell has an asterisk next to "In" ("In [*]:"), this means it is still processing</li>
<li>When the asterisk turns into any number, you can run the next cell</li></ul>

<h3>Import relevant python packages; Allow both Python 2 and 3</h3>

In [7]:
import os,sys
import requests
from bs4 import BeautifulSoup   # parser for html
import json, string, getpass

#to allow both Python 2 and 3
try:
    input = raw_input  
except NameError: #Python 3
    pass

<h3>Gets VAMPS username and password, then attempts login to VAMPS</h3>

In [8]:
#get VAMPS username and password
user = input("Enter your VAMPS username: ")
pw = getpass.getpass("Enter your VAMPS password: ")

conn = {'user': user,
        'passwd': pw,
         # vamps:             https://vamps2.mbl.edu
         # vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
         # localhost:         http://localhost:3000 
        'hosturl':'https://vamps2.mbl.edu'
       }

#attempt login to VAMPS
s = requests.Session()
r = s.post(conn['hosturl']+'/users/login', data={'username':conn['user'], 'password':conn['passwd']})

Enter your VAMPS username: ruzics
Enter your VAMPS password: ········


<h3>If username/password is incorrect, exit program</h3>

In [9]:
#exit program if login unsuccessful
if r.url == 'https://vamps2.mbl.edu/users/login':
    sys.exit('Login not successful')
elif r.url == 'https://vamps2.mbl.edu/':
    print('Login successful')

Login successful


<h3>Set original configuration file, using MVCO eukaryote data as project</h3>

In [10]:
#original config; Emily B.'s MVCO eukaryote data
config = {
        "api":"1",
        "source":"VAMPS-API",
        "update_data":1,
        "normalization":"none",              
        "selected_distance":"morisita-horn",  
        "tax_depth":"phylum",                 
        "domains":["Archaea","Bacteria","Eukarya","Organelle","Unknown"],   
        "include_nas":"yes",                  
        "min_range":0,                        
        "max_range":100,                      

          # Emily B.'s MVCO eukaryote data
        'project':'MVCO_ciliate_timeseries2',   

          # Alpha Diversity visualization
        'image':'adiversity'
    }

<h3>Get and display project IDs</h3>

In [11]:
# get project ids:
r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.text
print(config['ds_order'])

["353191","353192","353193","353194","353195","353196","353197","353198","353199","353200","353201","353202","353203","353204","353205","353206","353207","353208","353209","353210","353211","353212","353213","353214","353215","353216","353217"]


<h3>Create remote configuration and get timestamp (file_prefix)</h3>

In [12]:
# Get timestamp (filename prefix):
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string

<h3>Save matrix file which is integral to VAMPS images</h3>

In [13]:
import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)

<h3>Save image file</h3>

In [14]:
r = s.post(conn['hosturl']+'/api/create_image', timeout=30, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
print('Done writing local file:',local_filename)

ruzics_1501776915240-adiversity-api.csv
Done writing local file: ruzics_1501776915240-adiversity-api.csv


<h3>Show Alpha Diversity output for MVCO eukaryote data</h3>

In [15]:
from IPython.core.display import display, HTML
out = ''
out = HTML("<style>.container { width:100% !important; }</style>"+return_result)
out

Dataset,observed richness,ACE,chao1,Shannon,Simpson
MVCO_ciliate_timeseries2--RG10join,3,4.125,3.0,0.00104311787583,0.0001297636222
MVCO_ciliate_timeseries2--RG11join,6,12.0,7.5,0.04560676973,0.00951792891686
MVCO_ciliate_timeseries2--RG12join,7,7.66666666667,7.0,0.0354457589366,0.00640060925154
MVCO_ciliate_timeseries2--RG13join,11,11.5204081633,11.0,0.260846073267,0.0568602997546
MVCO_ciliate_timeseries2--RG14join,3,3.0,3.0,0.00134989442722,0.000169181660708
MVCO_ciliate_timeseries2--RG15join,7,10.0106846063,10.0,0.00540072764325,0.000718990523862
MVCO_ciliate_timeseries2--RG16join,7,11.9393939394,10.0,0.00502377048831,0.000680516529676
MVCO_ciliate_timeseries2--RG17join,6,8.23140495868,7.0,0.00917918876043,0.00133804122773
MVCO_ciliate_timeseries2--RG18join,7,13.0,9.0,0.00670204393226,0.00088875304526
MVCO_ciliate_timeseries2--RG19join,3,3.0,3.0,0.0035240349061,0.000495072461672


<h3>Change configuration file to be for MVCO bacteria data, keeping everything else the same</h3>

In [16]:
#change config file project; Kristen's MVCO bacteria data
config['project'] = 'AFP_MVCO_Bv6'

<h3>Now use the same method as before to produce the visualization, first getting and displaying dataset IDs</h3>

In [17]:
# get project ids:
r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.text
print(config['ds_order'])

[336411,336422,336433,336444,336455,336460,336461,336462,336463,336412,336413,336414,336415,336416,336418,336417,336419,336420,336421,336423,336424,336425,336426,336427,336428,336429,336430,336431,336432,336434,336435,336436,336437,336438,336439,336440,336441,336442,336443,336445,336446,336447,336448,336449,336450,336451,336452,336453,336454,336456,336457,336458,336459,336408,336409,336407,336410]


<h3>Create remote configuration and get timestamp (file_prefix)</h3>

In [18]:
# Get timestamp (filename prefix):
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string

<h3>Save matrix file which is integral to VAMPS images</h3>

In [19]:
import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)

<h3>Save image file</h3>

In [20]:
r = s.post(conn['hosturl']+'/api/create_image', timeout=30, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
print('Done writing local file:',local_filename)

ruzics_1501776941950-adiversity-api.csv
Done writing local file: ruzics_1501776941950-adiversity-api.csv


<h3>Show Alpha Diversity output for MVCO bacteria data</h3>

In [21]:
out = ''
out = HTML("<style>.container { width:100% !important; }</style>"+return_result)
out

Dataset,observed richness,ACE,chao1,Shannon,Simpson
AFP_MVCO_Bv6--MVCO_254,31,35.613592166,34.3333333333,1.62536773265,0.51890790385
AFP_MVCO_Bv6--MVCO_257,30,30.5777777778,30.5,2.18848485814,0.712567822549
AFP_MVCO_Bv6--MVCO_258,29,30.4784780024,30.5,2.1382332799,0.694620929592
AFP_MVCO_Bv6--MVCO_259,31,31.3949879315,31.0,2.24160659536,0.719122841686
AFP_MVCO_Bv6--MVCO_260,29,29.5880233918,30.0,2.26941822502,0.730933965492
AFP_MVCO_Bv6--MVCO_261,32,35.0552832244,35.0,2.26547491787,0.723606218721
AFP_MVCO_Bv6--MVCO_262,31,31.2524594041,31.0,2.23801458889,0.728176538865
AFP_MVCO_Bv6--MVCO_263,32,34.4436181252,35.0,2.08951530054,0.684919019435
AFP_MVCO_Bv6--MVCO_265,29,29.3999047189,30.0,2.0223078724,0.676434599631
AFP_MVCO_Bv6--MVCO_266,29,29.7888791601,29.3333333333,2.12283215978,0.695957050951
